In [3]:
!pip install dart-fss

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/41.7 kB ? eta -:--:--
     -------------------------------------- 41.7/41.7 kB 983.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/145.5 kB ? eta -:--:--
   --------------------------------- ------ 122.9/145.5 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 145.5/145.5 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
    --------------------------------------- 0.2/8.8 MB 5.0 MB/s eta 0:00:02
   - -------------------------------------- 0.3/8.8 MB 4.4 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/8.8 MB 4.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/8.8 MB 4.1 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/8.8 MB 3.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.8 MB 4.3 MB/s et

In [6]:
import pandas as pd
import numpy as np
import requests
import json
import zipfile
import xmltodict
import time
import warnings
import re
import xmltodict
from tqdm import tqdm
from io import BytesIO

warnings.filterwarnings(action='ignore')

In [8]:
url = 'https://opendart.fss.or.kr/api/corpCode.xml'
res = requests.get(url, params={'crtfc_key': api_key})
z = zipfile.ZipFile(BytesIO(res.content))

data_xml = z.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data_dart_1 = pd.DataFrame(data_dict.get('result', {}).get('list'))
data_dart_1

data_dart_1_nn = data_dart_1[data_dart_1.stock_code.notnull()]
data_dart_1_nn

,corp_code,corp_name,stock_code,modify_date
2006,00260985,한빛네트,036720,20170630
2018,00264529,엔플렉스,040130,20170630
2019,00358545,동서정보기술,055000,20170630
2781,00231567,애드모바일,032600,20170630
3884,00247939,씨모스,037600,20170630
...,...,...,...,...
103848,00126414,삼성제약,001360,20231005
103860,00116426,코센,009730,20231205
103861,00107987,남해화학,025860,20231205
103862,01068658,디딤이앤에프,217620,20231206


In [28]:
import dart_fss as dart
import pandas as pd

# API 키 등록
api_key = ''
dart.set_api_key(api_key=api_key)

# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

def get_reports(stock_code:str="005930"):
    
    # 종목 코드를 바탕으로 회사 찾기
    company = corp_list.find_by_stock_code(stock_code=stock_code)

    # 사업보고서 검색
    # 사업보고서 : a01, 반기보고서 : a02, 분기보고서 : a03
    reports = []
    
    # 1분기 보고서
    reports.append(company.search_filings(bgn_de="20230301", end_de="20230630", pblntf_detail_ty='a003'))
    
    # 2분기 보고서
    reports.append(company.search_filings(bgn_de="20230701", end_de="20230930", pblntf_detail_ty='a002'))
    
    # 3분기 보고서
    reports.append(company.search_filings(bgn_de="20231001", end_de="20231231", pblntf_detail_ty='a003'))
    
    return reports


In [29]:
reports = get_reports()

In [30]:
print(reports)

[{'page_count': 10,
 'page_no': 1,
 'report_list': [{'corp_cls': 'Y',
                  'corp_code': '00126380',
                  'corp_name': '삼성전자',
                  'flr_nm': '삼성전자',
                  'rcept_dt': '20230515',
                  'rcp_no': '20230515002335',
                  'report_nm': '분기보고서 (2023.03)',
                  'rm': '',
                  'stock_code': '005930'}],
 'total_count': 1,
 'total_page': 1}, {'page_count': 10,
 'page_no': 1,
 'report_list': [{'corp_cls': 'Y',
                  'corp_code': '00126380',
                  'corp_name': '삼성전자',
                  'flr_nm': '삼성전자',
                  'rcept_dt': '20230814',
                  'rcp_no': '20230814002534',
                  'report_nm': '반기보고서 (2023.06)',
                  'rm': '',
                  'stock_code': '005930'}],
 'total_count': 1,
 'total_page': 1}, {'page_count': 10,
 'page_no': 1,
 'report_list': [{'corp_cls': 'Y',
                  'corp_code': '00126380',
                 

In [34]:
for idx, report in enumerate(reports):
    # 리포트의 xbrl 데이터
    xbrl = report[0].xbrl
    
    # 연결재무제표 존재 여부 확인( True / False)
    xbrl.exist_consolidated()
    
    # 감사 정보 (영문) -> DataFrame 형태로 반환됨
    audit = xbrl.get_audit_information(lang='ko')
    
    # 연결 현금흐름표 추출 (리스트 반환)
    cf = xbrl.get_financial_statement()
    cf1 = xbrl.get_cash_flows()

    print(f"<{idx+1}분기 보고서 출력 진행>")
    print(cf)
    print()
    print(cf1)
    print()
    print()
    

<1분기 보고서 출력 진행>
[{'code': 'D210000', 'definition': '[D210000] Statement of financial position, current/non-current - Consolidated financial statements'}]

[{'code': 'D520000', 'definition': '[D520000] Statement of cash flows, indirect method - Consolidated financial statements'}]


<2분기 보고서 출력 진행>
[{'code': 'D210000', 'definition': '[D210000] Statement of financial position, current/non-current - Consolidated financial statements'}]

[{'code': 'D520000', 'definition': '[D520000] Statement of cash flows, indirect method - Consolidated financial statements'}]


<3분기 보고서 출력 진행>
[{'code': 'D210000', 'definition': '[D210000] 재무상태표, 유동/비유동법 - 연결 | Statement of financial position, current/non-current - Consolidated financial statements'}]

[{'code': 'D520000', 'definition': '[D520000] 현금흐름표, 간접법 - 연결 | Statement of cash flows, indirect method - Consolidated financial statements'}]




In [ ]:
# 연결 현금프름표
cf = cf[0]
cf1 = cf1[0]
# Pandas DataFrame으로 변환
df = cf.to_DataFrame()
df2 = cf1.to_DataFrame()

# Pandas DataFrame으로 변환 / 분류 정보 제외
df1_wo_class = cf.to_DataFrame(show_class=False)
df2_wo_class = cf1.to_DataFrame(show_class = False)
#print(df_wo_class)

df_wo_class = pd.concat([df1_wo_class, df2_wo_class], axis= 0 , ignore_index= True)
display(df_wo_class)
#df_wo_class.to_csv(r'C:\Users\user\OneDrive\바탕 화면\db 공모전/sample4.csv', index=False, encoding='euc-kr')